# Import Libraries

In [2]:
from openai import OpenAI
import pandas as pd
import asyncio
from openai import AsyncOpenAI
from tqdm import tqdm
import os
import json

from llm import get_top5


In [4]:
import requests
import os

# Define the API key
openai_api = os.getenv("OPENAI_API_KEY")

# Set the headers with the API key
headers = {
	"Authorization": f"Bearer {openai_api}"
}

# Check model list
response = requests.get("https://staging-litellm.moneylion.io/models", headers=headers)
print(response.json())

{'data': [{'id': 'gpt-3.5-turbo-0125', 'object': 'model', 'created': 1677610602, 'owned_by': 'openai'}, {'id': 'gpt-4o-mini', 'object': 'model', 'created': 1677610602, 'owned_by': 'openai'}], 'object': 'list'}


# Read Data

In [115]:
df_1000 = pd.read_csv('data/1000_users_latest_credit.csv')
df_1000.columns = df_1000.columns.str.lower()

In [7]:
df_1000['vantage_score_category'] = pd.cut(df_1000['vantage_score3'],
                                           bins=[-float('inf'), 599, 660, 715, 747, 850, float('inf')],
                                           labels=['Poor', 'Fair', 'Good', 'Very Good', 'Exceptional', 'Out of Range'])

In [5]:
df_1000.groupby('vantage_score_category').size()

/var/folders/wf/dnb0jm9x1vnc27qxnk9yr6yh0000gp/T/ipykernel_19449/553982992.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_1000.groupby('vantage_score_category').size()


vantage_score_category
Poor            728
Fair            191
Good             66
Very Good         9
Exceptional       6
Out of Range      0
dtype: int64

In [17]:
import json

first_row_json = df_1000.iloc[0].to_json()
first_row_dict = json.loads(first_row_json)
first_row_dict['vantage_score3']

460

# sample groundtruth set

In [83]:
# Sample rows from each category
sampled_df = pd.concat([
    df_1000[df_1000['vantage_score_category'] == 'Poor'].sample(10, random_state=1),
    df_1000[df_1000['vantage_score_category'] == 'Fair'].sample(3, random_state=1),
    df_1000[df_1000['vantage_score_category'] == 'Good'].sample(1, random_state=1),
    df_1000[df_1000['vantage_score_category'] == 'Very Good'].sample(1, random_state=1),
    df_1000[df_1000['vantage_score_category'] == 'Exceptional'].sample(1, random_state=1)
])
sampled_df.groupby('vantage_score_category').size()

/var/folders/wf/dnb0jm9x1vnc27qxnk9yr6yh0000gp/T/ipykernel_16448/596256927.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  sampled_df.groupby('vantage_score_category').size()


vantage_score_category
Poor            10
Fair             3
Good             1
Very Good        1
Exceptional      1
Out of Range     0
dtype: int64

In [7]:
sampled_df = df_1000[df_1000['user_email']=='399777dcde15a99f492a3f21b4b26e5e3127d2512f71e0cb0e6f568d7eb43e06']

In [8]:
first_row_dict = json.loads(sampled_df[sampled_df['vantage_score_category'] == 'Poor'].iloc[0].to_json())

# Prompt Building

In [74]:
credit_score = first_row_dict['vantage_score3']
ontime_payment_pct = first_row_dict['snap_shot__on_time_payment_percentage']
late_payment_pct = first_row_dict['snap_shot__late_payment_percentage']
delinquent_accounts = first_row_dict['snap_shot__delinquent_accounts']
derogatory_accounts = first_row_dict['snap_shot__derogatory_accounts']
utilization = first_row_dict['snap_shot__utilization']
number_of_inquiries = first_row_dict['snap_shot__number_of_inquiries']
available_credit = first_row_dict['snap_shot__available_credit']
date_of_oldest_trade = first_row_dict['snap_shot__date_of_oldest_trade']
age_of_credit = first_row_dict['snap_shot__age_of_credit']
total_installment = first_row_dict['snap_shot__total_open_installment_accounts']
total_revolving = first_row_dict['snap_shot__total_open_revolving_accounts']
total_balances = first_row_dict['snap_shot__total_balances']
installment_balances = first_row_dict['snap_shot__balance_open_installment_accounts']
revolving_balances = first_row_dict['snap_shot__balance_open_revolving_accounts']
mortgage_balances = first_row_dict['snap_shot__balance_open_mortgage_accounts']
other_balances = first_row_dict['snap_shot__balance_open_other_accounts']
collection_balances = first_row_dict['snap_shot__balance_open_collection_accounts']

## Attempt 1: `gpt-3.5-turbo` zero shot, CoT

In [72]:
system_prompt = """
You are an expert financial advisor AI, specializing in personal credit improvement. 
Your role is to analyze any credit-related data the user provides and then generate 
a clear, concise set of recommendations. 

- You will receive a user prompt containing specific credit data fields. 
- You should use chain-of-thought reasoning internally to arrive at your conclusion, 
  but only provide a concise and understandable explanation of your final recommendations 
  in the answer (i.e., do not expose all intermediate reasoning steps verbatim). 
- Your final output should be organized, practical, and directly address how the user 
  can improve their credit score based on the data provided.

Remember:
1. Stay on topic: The conversation is about credit improvement advice. 
2. Provide clear, actionable steps in your response, referencing the user’s data. 
3. If the user asks for chain-of-thought, provide a summarized or ‘mini’ version 
   rather than full internal reasoning.

"""

In [75]:
user_prompt = f"""
Below is my credit data. Please analyze it step by step using chain-of-thought reasoning 
and produce a concise set of personalized recommendations that explain what actions 
I can take to improve my credit score. 
For each recommendation, briefly mention how it ties back to the relevant data. 

Here is the data:

Credit Score: {credit_score}
Payment History:
- On-Time Payment Percentage: {ontime_payment_pct}
- Late Payment Percentage: {late_payment_pct}
- No. of Delinquent Accounts: {delinquent_accounts}
- No. of Derogatory Accounts: {derogatory_accounts}
Credit Utilization:
- Credit Utilization: {utilization}
- Total Balances: {total_balances}
- Total Available Credit: {available_credit}
- Installment Balances: {installment_balances}
- Revolving Balances: {revolving_balances}
Length of Credit History:
- Date of Oldest Trade: {date_of_oldest_trade}
- Age of Credit in Years: {age_of_credit}
Credit Mix:
- Total Open Installment Accounts: {total_installment}
- Total Open Revolving Accounts: {total_revolving}


### Instructions:
1. First, identify the user’s critical issues that most affect the credit score. 
2. Next, propose at least three prioritized recommendations on how to address these issues. 
3. Provide a short explanation for each recommendation, connecting it to the specific data. 
4. Present the final answer in a concise, actionable format.

Please begin by laying out your thought process briefly (chain-of-thought style), 
and then summarize your final recommendations.

"""

## Attempt 2: gpt-3.5-turbo, specify format given by o1 model

In [112]:
system_prompt = """
Please analyze the credit score data provided above and generate a concise, user-friendly explanation of the credit situation. Follow the format:

Score Health
Key Negative Factors
Positive Factors (or Less Negative)
What Might Help Improve the Score
Conclusion

Focus on late payments, collections, the lack of open revolving accounts, and other important factors that contribute to a VantageScore 3. 
Provide clear, actionable steps to help improve the score. 
"""

In [113]:
user_prompt = """
You are a credit analysis assistant. You are given a detailed credit report and must provide an expert summary and recommendations in a specific, structured format. 

You must:
1. Use the credit score data to inform your analysis. 
2. Clearly highlight any issues that could be impacting the user’s credit score (e.g., delinquent accounts, late payments, collections).
3. List any positive or less negative elements (e.g., age of credit, on-time payments, account diversity).
4. Provide recommendations for improving the score, focusing on actionable steps (e.g., addressing collections, making on-time payments, adding a secured card).
5. Write concisely in plain language. 
6. Do not reveal or explain your internal chain of thought. Present only the final written content.

Here is the credit score data you will use for your analysis:

{
  "_id": "66ec502cc0b33822acefb102",
  "user_id": "6515c12f1ddcd17a423751ee",
  "user_email": "399777dcde15a99f492a3f21b4b26e5e3127d2512f71e0cb0e6f568d7eb43e06",
  "created_at": "2024-09-19 16:24:12.712",
  "credit_report_frozen": false,
  "deleted": false,
  "is_enrolled": true,
  "updated_at": "2025-02-21 07:30:59.182",
  "population_rank": 10,
  "snap_shot__open_accounts": 6,
  "snap_shot__total_closed_accounts": null,
  "snap_shot__closed_accounts": 6,
  "snap_shot__total_accounts": 12,
  "snap_shot__delinquent_accounts": 2,
  "snap_shot__derogatory_accounts": 4,
  "snap_shot__total_balances": 12802.0,
  "snap_shot__total_monthly_payments": 291,
  "snap_shot__number_of_inquiries": 5,
  "snap_shot__total_public_records": 0,
  "snap_shot__available_credit": 0.0,
  "snap_shot__utilization": 0.0,
  "snap_shot__date_of_oldest_trade": "2020-04-08",
  "snap_shot__age_of_credit": 4.9,
  "snap_shot__total_open_installment_accounts": 1,
  "snap_shot__total_open_collection_accounts": 3,
  "snap_shot__total_open_mortgage_accounts": 0,
  "snap_shot__total_open_revolving_accounts": 0,
  "snap_shot__total_open_other_accounts": 2,
  "snap_shot__balance_open_installment_accounts": 8080,
  "snap_shot__balance_open_collection_accounts": 4491,
  "snap_shot__balance_open_mortgage_accounts": 0,
  "snap_shot__balance_open_revolving_accounts": 0,
  "snap_shot__balance_open_other_accounts": 0,
  "snap_shot__on_time_payment_percentage": "77.78%",
  "snap_shot__late_payment_percentage": "22.22%",
  "thin_file__description": null,
  "vantage_score3": 517,
  "vantage_score_category": "Poor"
}

Ensure the final response includes all pertinent details and recommendations in the specified format.
"""

In [114]:
print(user_prompt)


You are a credit analysis assistant. You are given a detailed credit report and must provide an expert summary and recommendations in a specific, structured format. 

You must:
1. Use the credit score data to inform your analysis. 
2. Clearly highlight any issues that could be impacting the user’s credit score (e.g., delinquent accounts, late payments, collections).
3. List any positive or less negative elements (e.g., age of credit, on-time payments, account diversity).
4. Provide recommendations for improving the score, focusing on actionable steps (e.g., addressing collections, making on-time payments, adding a secured card).
5. Write concisely in plain language. 
6. Do not reveal or explain your internal chain of thought. Present only the final written content.

Here is the credit score data you will use for your analysis:

{
  "_id": "66ec502cc0b33822acefb102",
  "user_id": "6515c12f1ddcd17a423751ee",
  "user_email": "399777dcde15a99f492a3f21b4b26e5e3127d2512f71e0cb0e6f568d7eb43e

In [119]:
response = get_top5(system_prompt,user_prompt,model='gpt-3.5-turbo-0125')

In [120]:
print(response.choices[0].message.content)

Credit Score Analysis:

Score Health: Poor

Key Negative Factors:
1. Delinquent Accounts: 2 delinquent accounts impacting the score.
2. Lack of Open Revolving Accounts: No open revolving accounts which can affect credit utilization.
3. Late Payment Percentage: 22.22% late payment rate.

Positive Factors (or Less Negative):
1. On-Time Payment Percentage: 77.78% on-time payments demonstrate some responsible credit behavior.
2. Age of Credit: Credit accounts have an average age of 4.9 years, which is relatively stable.

What Might Help Improve the Score:
1. Address Delinquent Accounts: Focus on resolving the 2 delinquent accounts to stop further negative impact.
2. Establish Revolving Accounts: Consider opening a secured credit card to improve credit utilization.
3. Improve Payment Timeliness: Strive to make all future payments on time to reduce the late payment percentage.

Conclusion:
The credit score of 517 is currently categorized as poor due to delinquent accounts, late payments, and

## Attempt 3: gpt-4o-mini, format by O1

In [117]:
response = get_top5(system_prompt,user_prompt,model='gpt-4o-mini')

In [118]:
print(response.choices[0].message.content)

**Score Health**  
Your current VantageScore 3 is 517, which falls into the "Poor" category. This indicates significant challenges in your credit profile.

**Key Negative Factors**  
- **Late Payments**: 22.22% of your payments are late, which directly impacts your score negatively.
- **Delinquent Accounts**: You have 2 delinquent accounts that are affecting your credit standing.
- **Collections**: There are 3 open collection accounts with a total balance of $4,491.
- **Lack of Revolving Accounts**: You have no open revolving accounts, which is crucial for improving your score and credit mix.

**Positive Factors (or Less Negative)**  
- **Number of Accounts**: You have a total of 12 accounts, which shows some diversity.
- **Age of Credit**: Your average credit age is 4.9 years, indicating some credit history.
- **Closed Accounts**: Having a closed account history (6 closed accounts) may not be negative, depending on how they were managed.

**What Might Help Improve the Score**  
1. **M

# JSON mode

## Attempt 4: gpt-4o-mini, json mode

In [10]:
system_prompt = """
Please analyze the credit score data provided above and generate a concise, user-friendly explanation of the credit situation. 
Your response must be in valid JSON format and include only the following keys:
{
  "Score Health": "",
  "Key Negative Factors": [],
  "Positive Factors (or Less Negative)": [],
  "What Might Help Improve the Score": [],
  "Conclusion": ""
}

Focus on late payments, collections, the lack of open revolving accounts, and other important factors that contribute to a VantageScore 3. 
Provide clear, actionable steps to help improve the score. 
"""

In [11]:
user_prompt = """
You are a credit analysis assistant. You are given a detailed credit report and must provide an expert summary and recommendations in a specific, structured format. 

You must:
1. Use the credit score data to inform your analysis. 
2. Clearly highlight any issues that could be impacting the user’s credit score (e.g., delinquent accounts, late payments, collections).
3. List any positive or less negative elements (e.g., age of credit, on-time payments, account diversity).
4. Provide recommendations for improving the score, focusing on actionable steps (e.g., addressing collections, making on-time payments, adding a secured card).
5. Write concisely in plain language. 
6. Do not reveal or explain your internal chain of thought. Present only the final written content.

Here is the credit score data you will use for your analysis:

{
  "_id": "66ec502cc0b33822acefb102",
  "user_id": "6515c12f1ddcd17a423751ee",
  "user_email": "399777dcde15a99f492a3f21b4b26e5e3127d2512f71e0cb0e6f568d7eb43e06",
  "created_at": "2024-09-19 16:24:12.712",
  "credit_report_frozen": false,
  "deleted": false,
  "is_enrolled": true,
  "updated_at": "2025-02-21 07:30:59.182",
  "population_rank": 10,
  "snap_shot__open_accounts": 6,
  "snap_shot__total_closed_accounts": null,
  "snap_shot__closed_accounts": 6,
  "snap_shot__total_accounts": 12,
  "snap_shot__delinquent_accounts": 2,
  "snap_shot__derogatory_accounts": 4,
  "snap_shot__total_balances": 12802.0,
  "snap_shot__total_monthly_payments": 291,
  "snap_shot__number_of_inquiries": 5,
  "snap_shot__total_public_records": 0,
  "snap_shot__available_credit": 0.0,
  "snap_shot__utilization": 0.0,
  "snap_shot__date_of_oldest_trade": "2020-04-08",
  "snap_shot__age_of_credit": 4.9,
  "snap_shot__total_open_installment_accounts": 1,
  "snap_shot__total_open_collection_accounts": 3,
  "snap_shot__total_open_mortgage_accounts": 0,
  "snap_shot__total_open_revolving_accounts": 0,
  "snap_shot__total_open_other_accounts": 2,
  "snap_shot__balance_open_installment_accounts": 8080,
  "snap_shot__balance_open_collection_accounts": 4491,
  "snap_shot__balance_open_mortgage_accounts": 0,
  "snap_shot__balance_open_revolving_accounts": 0,
  "snap_shot__balance_open_other_accounts": 0,
  "snap_shot__on_time_payment_percentage": "77.78%",
  "snap_shot__late_payment_percentage": "22.22%",
  "thin_file__description": null,
  "vantage_score3": 517,
  "vantage_score_category": "Poor"
}

Ensure the final response includes all pertinent details and recommendations in the specified format.
"""

In [12]:
response = get_top5(system_prompt,user_prompt,model='gpt-4o-mini')
print(response.choices[0].message.content)

{
  "Score Health": "Poor",
  "Key Negative Factors": [
    "Delinquent accounts: 2",
    "Derogatory accounts: 4",
    "Late payment percentage: 22.22%",
    "No open revolving accounts",
    "High balance on collection accounts"
  ],
  "Positive Factors (or Less Negative)": [
    "Age of credit: 4.9 years",
    "On-time payment percentage: 77.78%",
    "Total accounts: 12"
  ],
  "What Might Help Improve the Score": [
    "Address delinquent and derogatory accounts by making payments or negotiating with creditors",
    "Establish one or more open revolving accounts, such as a secured credit card",
    "Reduce the balances on collection accounts through repayment plans",
    "Ensure all future payments are made on time to improve on-time payment percentage"
  ],
  "Conclusion": "Your credit score is currently poor due to a combination of delinquent accounts, derogatory marks, and the absence of revolving credit. Focusing on addressing negative accounts and building a positive credit h

In [13]:
result_json = json.loads(response.choices[0].message.content)
result_json['Score Health']

'Poor'

## Attempt 5: gpt-4o-mini, json mode, add instructions to get better response

In [50]:
system_prompt = """
You are a warm, empathetic credit analysis with expertise in VantageScore 3.0. You are given credit score data and must produce a concise, user-friendly explanation of the credit situation in valid JSON. 
The style of your response should be encouraging and supportive, while still communicating important details and actionable steps. 

Your response must use exactly the following JSON structure and keys (no extra keys or nesting):
{
  "Score Health": "",
  "Key Negative Factors": [],
  "Positive Factors (or Less Negative)": [],
  "What Might Help Improve the Score": [],
  "Conclusion": "",
  "Summary": ""
}

Important:
1. Keep the JSON valid—no additional fields or text outside of the JSON object.
2. Use an empathetic yet clear tone. 
3. Do not just give a value. Elaborate each point in a concise way
4. Provide specific details about factors affecting credit score and easy to understand explanation.
5. Focus more on factors that have greater impact on credit score
6. in each factor, give the factor values before writing the explanation
7. Offer actionable recommendations in the "What Might Help Improve the Score" section.
8. Keep each list entry short but supportive, avoiding overly dry or blunt language, and elaborate each point in a concise way on why is the factor important
9. Focus on empowering the user to take steps that improve their credit score.

"""

In [51]:
user_prompt = """
Analyze the following credit score data and produce an output in valid JSON format (using the keys defined in the system prompt). Incorporate an empathetic, encouraging tone while providing a detailed breakdown of the credit situation and clear recommendations:

{
  "_id": "66ec502cc0b33822acefb102",
  "user_id": "6515c12f1ddcd17a423751ee",
  "user_email": "399777dcde15a99f492a3f21b4b26e5e3127d2512f71e0cb0e6f568d7eb43e06",
  "created_at": "2024-09-19 16:24:12.712",
  "credit_report_frozen": false,
  "deleted": false,
  "is_enrolled": true,
  "updated_at": "2025-02-21 07:30:59.182",
  "population_rank": 10,
  "snap_shot__open_accounts": 6,
  "snap_shot__total_closed_accounts": null,
  "snap_shot__closed_accounts": 6,
  "snap_shot__total_accounts": 12,
  "snap_shot__delinquent_accounts": 2,
  "snap_shot__derogatory_accounts": 4,
  "snap_shot__total_balances": 12802.0,
  "snap_shot__total_monthly_payments": 291,
  "snap_shot__number_of_inquiries": 5,
  "snap_shot__total_public_records": 0,
  "snap_shot__available_credit": 0.0,
  "snap_shot__utilization": 0.0,
  "snap_shot__date_of_oldest_trade": "2020-04-08",
  "snap_shot__age_of_credit": 4.9,
  "snap_shot__total_open_installment_accounts": 1,
  "snap_shot__total_open_collection_accounts": 3,
  "snap_shot__total_open_mortgage_accounts": 0,
  "snap_shot__total_open_revolving_accounts": 0,
  "snap_shot__total_open_other_accounts": 2,
  "snap_shot__balance_open_installment_accounts": 8080,
  "snap_shot__balance_open_collection_accounts": 4491,
  "snap_shot__balance_open_mortgage_accounts": 0,
  "snap_shot__balance_open_revolving_accounts": 0,
  "snap_shot__balance_open_other_accounts": 0,
  "snap_shot__on_time_payment_percentage": "77.78%",
  "snap_shot__late_payment_percentage": "22.22%",
  "thin_file__description": null,
  "vantage_score3": 517,
  "vantage_score_category": "Poor"
}

Please ensure your final answer is supportive, understandable, and presented solely as a JSON object with the specified structure.
"""

In [52]:
response = get_top5(system_prompt,user_prompt,model='gpt-4o-mini')
print(response.choices[0].message.content)

{
  "Score Health": "Your current credit score is 517, placing you in the 'Poor' category. This indicates there are several areas for improvement we can work on together.",
  "Key Negative Factors": [
    "Delinquent Accounts: 2 - These accounts have not been paid as agreed, which negatively impacts your score.",
    "Derogatory Accounts: 4 - These accounts reflect serious issues like defaults or charge-offs, making it harder to get new credit.",
    "High Balance on Installment Accounts: $8,080 - A high outstanding balance can suggest a risk to lenders.",
    "Late Payment Percentage: 22.22% - Missing payments can significantly hurt your score, as it reflects negatively on your credit responsibility."
  ],
  "Positive Factors (or Less Negative)": [
    "On-Time Payment Percentage: 77.78% - You have managed to make most of your payments on time, which is a great foundation to build on.",
    "Credit Age: 4.9 years - A longer credit history can work in your favor, especially if you keep

In [53]:
result_json = json.loads(response.choices[0].message.content)
# result_json['Key Negative Factors']
result_json

{'Score Health': "Your current credit score is 517, placing you in the 'Poor' category. This indicates there are several areas for improvement we can work on together.",
 'Key Negative Factors': ['Delinquent Accounts: 2 - These accounts have not been paid as agreed, which negatively impacts your score.',
  'Derogatory Accounts: 4 - These accounts reflect serious issues like defaults or charge-offs, making it harder to get new credit.',
  'High Balance on Installment Accounts: $8,080 - A high outstanding balance can suggest a risk to lenders.',
  'Late Payment Percentage: 22.22% - Missing payments can significantly hurt your score, as it reflects negatively on your credit responsibility.'],
 'Positive Factors (or Less Negative)': ['On-Time Payment Percentage: 77.78% - You have managed to make most of your payments on time, which is a great foundation to build on.',
  'Credit Age: 4.9 years - A longer credit history can work in your favor, especially if you keep improving your payment ha

## Attempt 6: gpt-4o-mini, change section name

In [ ]:
system_prompt = """
You are a warm, empathetic credit analysis with expertise in VantageScore 3.0. You are given credit score data and must produce a concise, user-friendly explanation of the credit situation in valid JSON. 
The style of your response should be encouraging and supportive, while still communicating important details and actionable steps. 

Your response must use exactly the following JSON structure and keys (no extra keys or nesting):
{
  "Score Health": "",
  "What's dragging down your score?": [],
  "What's boosting your score?": [],
  "What might help improve the score?": [],
  "Conclusion": "",
  "Summary": ""
}

Important:
1. Keep the JSON valid—no additional fields or text outside of the JSON object.
2. Use an empathetic yet clear tone. 
3. Do not just give a value. Elaborate each point in a concise way
4. Provide specific details about factors affecting credit score and easy to understand explanation.
5. Focus more on factors that have greater impact on credit score
6. in each factor, give the factor values before writing the explanation
7. Offer actionable recommendations in the "What Might Help Improve the Score" section.
8. Keep each list entry short but supportive, avoiding overly dry or blunt language, and elaborate each point in a concise way on why is the factor important
9. Focus on empowering the user to take steps that improve their credit score.
"""

In [55]:
user_prompt = """
Analyze the following credit score data and produce an output in valid JSON format (using the keys defined in the system prompt). Incorporate an empathetic, encouraging tone while providing a detailed breakdown of the credit situation and clear recommendations:

{
  "_id": "66ec502cc0b33822acefb102",
  "user_id": "6515c12f1ddcd17a423751ee",
  "user_email": "399777dcde15a99f492a3f21b4b26e5e3127d2512f71e0cb0e6f568d7eb43e06",
  "created_at": "2024-09-19 16:24:12.712",
  "credit_report_frozen": false,
  "deleted": false,
  "is_enrolled": true,
  "updated_at": "2025-02-21 07:30:59.182",
  "population_rank": 10,
  "snap_shot__open_accounts": 6,
  "snap_shot__total_closed_accounts": null,
  "snap_shot__closed_accounts": 6,
  "snap_shot__total_accounts": 12,
  "snap_shot__delinquent_accounts": 2,
  "snap_shot__derogatory_accounts": 4,
  "snap_shot__total_balances": 12802.0,
  "snap_shot__total_monthly_payments": 291,
  "snap_shot__number_of_inquiries": 5,
  "snap_shot__total_public_records": 0,
  "snap_shot__available_credit": 0.0,
  "snap_shot__utilization": 0.0,
  "snap_shot__date_of_oldest_trade": "2020-04-08",
  "snap_shot__age_of_credit": 4.9,
  "snap_shot__total_open_installment_accounts": 1,
  "snap_shot__total_open_collection_accounts": 3,
  "snap_shot__total_open_mortgage_accounts": 0,
  "snap_shot__total_open_revolving_accounts": 0,
  "snap_shot__total_open_other_accounts": 2,
  "snap_shot__balance_open_installment_accounts": 8080,
  "snap_shot__balance_open_collection_accounts": 4491,
  "snap_shot__balance_open_mortgage_accounts": 0,
  "snap_shot__balance_open_revolving_accounts": 0,
  "snap_shot__balance_open_other_accounts": 0,
  "snap_shot__on_time_payment_percentage": "77.78%",
  "snap_shot__late_payment_percentage": "22.22%",
  "thin_file__description": null,
  "vantage_score3": 517,
  "vantage_score_category": "Poor"
}

Please ensure your final answer is supportive, understandable, and presented solely as a JSON object with the specified structure.
"""

In [57]:
response = get_top5(system_prompt,user_prompt,model='gpt-4o-mini')
print(response.choices[0].message.content)

{
  "Score Health": "Your current VantageScore is 517, which falls into the 'Poor' category. This indicates that there are significant challenges to overcome to improve your credit health. However, every step you take towards better credit is a positive move.",
  "What's dragging down your score?": [
    "Delinquent Accounts: 2 accounts. Having unpaid debts can seriously affect your score, signaling to lenders that you may struggle with repayments.",
    "Derogatory Accounts: 4 accounts. These negative marks suggest a history of late payments or defaults, making you appear a higher risk to lenders.",
    "Late Payment Percentage: 22.22%. Missing payments can have a considerable impact on your credit score, as payment history is one of the most critical factors in scoring models.",
    "No Available Credit: $0. This indicates a lack of open revolving accounts, which can limit your credit utilization and overall score potential."
  ],
  "What's boosting your score?": [
    "On-Time Payme

In [ ]:
result_json = json.loads(response.choices[0].message.content)
# result_json['Key Negative Factors']
result_json

{'Score Health': "Your current credit score is 517, placing you in the 'Poor' category. This indicates there are several areas for improvement we can work on together.",
 'Key Negative Factors': ['Delinquent Accounts: 2 - These accounts have not been paid as agreed, which negatively impacts your score.',
  'Derogatory Accounts: 4 - These accounts reflect serious issues like defaults or charge-offs, making it harder to get new credit.',
  'High Balance on Installment Accounts: $8,080 - A high outstanding balance can suggest a risk to lenders.',
  'Late Payment Percentage: 22.22% - Missing payments can significantly hurt your score, as it reflects negatively on your credit responsibility.'],
 'Positive Factors (or Less Negative)': ['On-Time Payment Percentage: 77.78% - You have managed to make most of your payments on time, which is a great foundation to build on.',
  'Credit Age: 4.9 years - A longer credit history can work in your favor, especially if you keep improving your payment ha

# 5 users

In [8]:
df_5 = df_1000[:5]
df_5_json = json.loads(df_5.to_json(orient='records'))

In [ ]:
from llm import get_credit_improvement, calculate_accuracy_metrics
responses = await get_credit_improvement(df_5_json)

100%|██████████| 5/5 [00:00<00:00, 2759.41it/s]


## post process output

Note that `post process output` and `define evaluation metrics` are using pandas dataframe for development purpose. The use of lambda function is not efficient in large scale data
<br></br>
A proper calculate metrics function is defined later in the `calculate metrics` section

In [11]:
# Convert responses into a list of dictionaries
response_dicts = []
for i in range(0, len(responses)):
    response_i = json.loads(responses[i].choices[0].message.content)
    response_i['user_id'] = df_5_json[i]['user_id']
    response_dicts.append(response_i)

# Convert the list of dictionaries into a pandas DataFrame
responses_df = pd.DataFrame(response_dicts)
# Perform a left join with df_1000 using 'user_id'
final_df = df_5.merge(responses_df, on='user_id', how='left')
final_df

,_id,user_id,user_email,created_at,credit_report_frozen,deleted,is_enrolled,updated_at,population_rank,snap_shot__open_accounts,...,snap_shot__late_payment_percentage,thin_file__description,vantage_score3,vantage_score_category,Score Health,What's dragging down your score?,What's boosting your score?,What might help improve the score?,Conclusion,Summary
0,66ec26e2c0b33822acecbca5,66ec25713804f60b406c5dca,a46b2854647b839cd1caaa912457b636a7a9c9373d3fb1...,2024-09-19 13:28:02.711,False,False,True,2025-02-21 07:27:23.232,2,5,...,12.5%,NaN,460,Poor,"Your current VantageScore 3.0 is 460, which fa...","[Delinquent Accounts: 1 account is delinquent,...",[On-time Payment Percentage: 87.5% of your pay...,[Address Delinquencies: Focus on catching up w...,Improving your credit score is a journey that ...,While there are factors currently holding your...
1,66ec271ed58ccf36a23748a4,627300afba62be4a28ca17ba,049a9f1d119ced7afb64fb9e8a7f8ce07189e7ba709996...,2024-09-19 13:29:02.255,False,False,True,2025-02-21 07:27:24.195,5,4,...,25%,NaN,499,Poor,"Your current VantageScore 3.0 is 499, which fa...",[Delinquent Accounts: 1 - Having a delinquent ...,[On-time Payment Percentage: 75% - This is a p...,[Reduce Credit Utilization: Focus on paying do...,While your current credit situation poses chal...,Consider focusing on reducing outstanding bala...
2,66ec272ed58ccf36a23749c7,66ec2608868a9622564d46bb,9bc82cf1f08d07cf0551c8cefbde213eff5ba76fed3f05...,2024-09-19 13:29:18.628,False,False,True,2025-02-21 07:28:42.751,34,18,...,6.06%,NaN,610,Fair,"Your current credit score is 610, which falls ...",[Derogatory accounts: 7 - These accounts can s...,[On-time payments: 93.94% - Consistently payin...,[Reduce credit utilization - Aim to pay down y...,"With a few dedicated efforts, you can enhance ...",Your score is in the Fair range at 610. Focus ...
3,66ec273ed58ccf36a2374a13,6429bb6efe264063091b97ed,e8973321e28d8a774df7bd9a965b5a9d16d7e88f2f9fdc...,2024-09-19 13:29:34.085,False,False,True,2025-02-21 07:28:43.556,20,5,...,11.11%,NaN,551,Poor,"Your current score is 551, which falls within ...",[Delinquent Accounts (2): Having accounts that...,[On-Time Payment Percentage (88.89%): Maintain...,[Reduce Delinquent Accounts: Focus on bringing...,While your score is currently in the 'Poor' ra...,You've established a solid foundation with a g...
4,66ec274f766dc80a49fad7a9,66ec250d3804f60b406b4e4e,2a2e8fdcdc9522ae6d5e95c0fddc648ccd2273f372e8dc...,2024-09-19 13:29:51.443,False,False,True,2025-02-21 07:28:44.413,8,2,...,50%,NaN,515,Poor,Poor,"[Having 3 derogatory accounts, Having 1 delinq...","[Total number of accounts (4), Age of oldest a...","[Pay down delinquent and collection accounts, ...",While your current credit situation has some c...,Your VantageScore of 515 places you in the 'Po...


## define evaluation metrics

In [42]:
# evaluation
# check if the right vantage_score3 is mentioned in `Score Health` column
final_df['evaluate_score'] = final_df['Score Health'].apply(lambda x: any(str(score) in x for score in final_df['vantage_score3'].astype(str)))
final_df['evaluate_delinquent'] = final_df.apply(lambda row: not(row['snap_shot__delinquent_accounts'] > 0 and 'delinquent' not in ' '.join(row["What's dragging down your score?"]).lower()), axis=1)
final_df['evaluate_derogatory'] = final_df.apply(lambda row: not(row['snap_shot__derogatory_accounts'] > 0 and 'derogatory' not in ' '.join(row["What's dragging down your score?"]).lower()), axis=1)
# convert late_payment_percentage to float and check if the right late_payment_percentage is mentioned in `What's dragging down your score?` column when late_payment_percentage > 0
final_df['late_payment_percentage_real'] = final_df['snap_shot__late_payment_percentage'].str.rstrip('%').astype('float') / 100.0
final_df['evaluate_late_payment'] = final_df.apply(lambda row: not(row['late_payment_percentage_real'] > 0 and 'late payment' not in ' '.join(row["What's dragging down your score?"]).lower()), axis=1)

In [53]:
# Combine all metrics into one table
metrics_df = final_df[['user_id', 'vantage_score3', 'evaluate_score', 'evaluate_delinquent', 'evaluate_derogatory', 'evaluate_late_payment']]

# Calculate the mean accuracy for each metric
accuracy_metrics = {
    'Metric': ['Vantage Score Accuracy', 'Delinquent Accounts Accuracy', 'Derogatory Accounts Accuracy', 'Late Payment Accuracy'],
    'Accuracy': [
        final_df['evaluate_score'].mean(),
        final_df['evaluate_delinquent'].mean(),
        final_df['evaluate_derogatory'].mean(),
        final_df['evaluate_late_payment'].mean()
    ]
}

accuracy_df = pd.DataFrame(accuracy_metrics)
accuracy_df

,Metric,Accuracy
0,Vantage Score Accuracy,0.8
1,Delinquent Accounts Accuracy,1.0
2,Derogatory Accounts Accuracy,1.0
3,Late Payment Accuracy,0.6


In [54]:
# checking inaccurate output
final_df[~final_df['evaluate_late_payment']]['What\'s dragging down your score?'].tolist()

[['Delinquent Accounts: 1 account is delinquent, meaning payments have been missed, which can significantly impact your score.',
  'Derogatory Accounts: 9 accounts have negative marks, suggesting serious issues that may deter lenders from extending credit.',
  'Number of Inquiries: You have 9 hard inquiries on your report, which can indicate a risky profile to lenders and lead to score penalties.',
  'Open Collection Accounts: 5 accounts are in collections, showing unpaid debts that negatively affect your credit standing.'],
 ['Having 3 derogatory accounts',
  'Having 1 delinquent account',
  '50% on-time payment history',
  'Recent credit inquiries (2)']]

## calculate metrics

In [155]:
def calculate_accuracy_metrics(responses, df_input):
    # Convert responses into a list of dictionaries
    response_dicts = []
    for i in range(len(responses)):
        response_i = json.loads(responses[i].choices[0].message.content)
        response_i['user_id'] = df_input[i]['user_id']
        response_dicts.append(response_i)

    # Initialize lists to store evaluation results
    evaluate_score = []
    evaluate_delinquent = []
    evaluate_derogatory = []
    evaluate_late_payment = []

    # Perform evaluation for each response
    for i, response in tqdm(enumerate(response_dicts)):
        user_data = df_input[i]
        score_health = response['Score Health']  # assume it's a string
        dragging_down_list = response["What's dragging down your score?"]
        # Check if 'dragging_down_list' is a list of dictionaries
        if isinstance(dragging_down_list, list) and all(isinstance(item, dict) for item in dragging_down_list):
            # Convert list of dictionaries to a single string with key-value pairs
            dragging_down = ' '.join([f"{key}: {value}" for item in dragging_down_list for key, value in item.items()]).lower()
        else:
            # Convert list of strings to a single string
            dragging_down = ' '.join(dragging_down_list).lower()
        # Evaluate if the correct VantageScore is mentioned in 'Score Health'
        evaluate_score.append(str(user_data['vantage_score3']) in score_health)
        
        # Evaluate if delinquent accounts are correctly mentioned
        evaluate_delinquent.append(
            not (user_data['snap_shot__delinquent_accounts'] > 0 and 'delinquent' not in dragging_down)
        )
        
        # Evaluate if derogatory accounts are correctly mentioned
        evaluate_derogatory.append(
            not (user_data['snap_shot__derogatory_accounts'] > 0 and 'derogatory' not in dragging_down)
        )
        
        # Handle empty percentage values and convert late payment percentage to float
        late_payment_percentage_str = user_data['snap_shot__late_payment_percentage']
        if late_payment_percentage_str in [None, "%"]:
            late_payment_percentage_real = 0.0
        else:
            late_payment_percentage_real = float(late_payment_percentage_str.rstrip('%')) / 100.0
        # Convert late payment percentage to float and evaluate if it's correctly mentioned
        evaluate_late_payment.append(
            not (late_payment_percentage_real > 0 and 'late payment' not in dragging_down)
        )

    # Calculate the mean accuracy for each metric
    accuracy_metrics = {
        'Metric': [
            'Vantage Score Accuracy',
            'Delinquent Accounts Accuracy',
            'Derogatory Accounts Accuracy',
            'Late Payment Accuracy'
        ],
        'Accuracy': [
            sum(evaluate_score) / len(evaluate_score),
            sum(evaluate_delinquent) / len(evaluate_delinquent),
            sum(evaluate_derogatory) / len(evaluate_derogatory),
            sum(evaluate_late_payment) / len(evaluate_late_payment)
        ]
    }

    # Return the accuracy metrics as a JSON string
    return json.dumps(accuracy_metrics, indent=4)


In [70]:
accuracy_metrics_json = calculate_accuracy_metrics(responses, df_5_json)
accuracy_metrics_dict = json.loads(accuracy_metrics_json)
pd.DataFrame(accuracy_metrics_dict)

,Metric,Accuracy
0,Vantage Score Accuracy,0.8
1,Delinquent Accounts Accuracy,1.0
2,Derogatory Accounts Accuracy,1.0
3,Late Payment Accuracy,0.6


# 1000 users

In [97]:
df_1000_json = json.loads(df_1000.to_json(orient='records'))

# Process df_1000_json in chunks of 50
chunk_size = 50
responses = []
for i in range(600, len(df_1000_json), chunk_size):
    chunk = df_1000_json[i:i + chunk_size]
    try:
        chunk_responses = await get_credit_improvement(chunk)
        responses.extend(chunk_responses)
    except asyncio.TimeoutError:
        print(f"Timeout error occurred for chunk starting at index {i}. Retrying...")
        chunk_responses = await get_credit_improvement(chunk)
        responses.extend(chunk_responses)


100%|██████████| 50/50 [00:00<00:00, 57550.82it/s]


In [99]:
# Combine response_bak and responses
combined_responses = responses_bak + responses

In [101]:
len(combined_responses)

1000

In [117]:
df_1000['snap_shot__late_payment_percentage'].value_counts()

snap_shot__late_payment_percentage
%         411
0          50
25%        36
14.29%     36
20%        32
         ... 
14.81%      1
20.93%      1
57.14%      1
4%          1
8.11%       1
Name: count, Length: 113, dtype: int64

In [ ]:
# Convert the combined_responses to a list of dictionaries
combined_responses_dicts = [response.choices[0].message.content for response in combined_responses]

# Write the list of dictionaries to a JSON file
with open('combined_responses.json', 'w') as f:
    json.dump(combined_responses_dicts, f, indent=4)

## run metrics calculation for 1000 users

In [157]:
accuracy_metrics_json = calculate_accuracy_metrics(combined_responses, df_1000_json)
accuracy_metrics_dict = json.loads(accuracy_metrics_json)
pd.DataFrame(accuracy_metrics_dict)

1000it [00:00, 164695.65it/s]


,Metric,Accuracy
0,Vantage Score Accuracy,0.821
1,Delinquent Accounts Accuracy,0.995
2,Derogatory Accounts Accuracy,0.979
3,Late Payment Accuracy,0.720


In [139]:
sample = json.loads(combined_responses[865].choices[0].message.content)
# dragging_down = ' '.join(sample["What's dragging down your score?"]).lower()
sample

{'Score Health': "Your current VantageScore 3.0 is 552, which falls into the 'Poor' category. This indicates there are significant areas for improvement, but with focused effort, you can increase your score.",
 "What's dragging down your score?": ["Derogatory Accounts: 2 - Having derogatory accounts indicates past due payments or collections, which considerably lowers your score. It's important to address these to improve your credit health.",
  'High Inquiries: 8 - Too many recent inquiries can signal risk to creditors. Consider limiting new credit applications in the near term to help your score recover.',
  "Available Credit: $0.00 - Without available credit, it can appear you have limited options for credit use, which isn't beneficial for your score. Building or maintaining available credit is key."],
 "What's boosting your score?": [],
 'What might help improve the score?': ['Address Derogatory Accounts - Contact creditors to negotiate payment plans or settlement options. Clearing